In [1]:
import pandas as pd
import ast
import data
'''
# 데이터 불러오기
data = pd.read_csv('./preprocessed-meta-California.csv')

# 'MISC' 컬럼과 'Category' 컬럼에서 nan 값을 처리하고 딕셔너리 형태의 데이터를 문자열로 변환
def safe_literal_eval(value):
    try:
        # 먼저 값이 nan인지 체크
        if pd.isna(value):
            return {}
        else:
            return ast.literal_eval(value)
    except ValueError:
        print("wrong value")
        return {}

data['MISC'] = data['MISC'].apply(safe_literal_eval)
data['category'] = data['category'].apply(lambda x: safe_literal_eval(x) if isinstance(x, str) else x)

# 원-핫 인코딩을 위한 임시 DataFrame 생성
encoded_data = pd.DataFrame()

# 각 레스토랑의 'MISC' 속성별로 원-핫 인코딩 수행
for attribute in ['Service options', 'Accessibility', 'Offerings', 'Amenities', 'Atmosphere']:
    # 속성 값을 추출하고 쉼표로 구분된 문자열로 변환
    attribute_values = data['MISC'].apply(lambda x: ','.join(x.get(attribute, [])))

    # 원-핫 인코딩 적용
    encoded = attribute_values.str.get_dummies(',')

    # 인코딩된 데이터를 임시 DataFrame에 추가
    encoded_data = pd.concat([encoded_data, encoded], axis=1)

# 'category' 열을 쉼표로 구분된 문자열로 변환
data['category'] = data['category'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# 원-핫 인코딩 적용
category_encoded = data['category'].str.get_dummies(sep=',')
encoded_data = pd.concat([encoded_data,category_encoded], axis=1)

non_zero_columns = encoded_data.columns[(encoded_data != 0).any(axis=0)]
encoded_data = encoded_data[non_zero_columns]


encoded_data.to_csv('./encoded_data.csv',index=False)
'''
encoded_data = pd.read_csv('./data/encoded_data.csv')
print(encoded_data)



       Curbside pickup  Delivery  Dine-in  Drive-through  In-store pick-up  \
0                    0         1        1              0                 0   
1                    1         1        1              0                 0   
2                    0         0        1              0                 0   
3                    0         0        1              0                 0   
4                    0         1        1              0                 0   
...                ...       ...      ...            ...               ...   
80631                0         1        1              0                 0   
80632                1         1        1              0                 0   
80633                1         1        1              0                 0   
80634                1         1        1              0                 0   
80635                1         1        1              1                 0   

       In-store pickup  In-store shopping  No-contact delivery 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 데이터 불러오기
data_2 = pd.read_csv('./data/restaurant_rating_dataset.csv')
data_2 = data_2.drop_duplicates(subset=['user', 'business', 'rating'])
data_2=data_2.head(1000)
pivot_data= data_2.pivot(index='user', columns='business', values='rating').fillna(0)


X = []
y = []

X = []
y = []

for user in pivot_data.index:
    user_profile = []
    user_ratings = []
    for business in pivot_data.columns:
        if not pd.isna(pivot_data.at[user, business]):
            # 사용자가 평가한 비즈니스의 특징 추출
            features = encoded_data.loc[business].tolist()
            user_profile.append(features)
            user_ratings.append(pivot_data.at[user, business])
    
    # 사용자 프로필을 평균하여 하나의 벡터로 만듦
    if user_profile:
        user_profile_avg = np.mean(user_profile, axis=0)
        X.append(user_profile_avg)
        # 사용자의 평균 평점 계산
        y.append(np.mean(user_ratings))

# 데이터프레임으로 변환
X = pd.DataFrame(X)
y = pd.Series(y)

# 데이터프레임으로 변환
X = pd.DataFrame(X)
y = pd.Series(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)






In [ ]:
# ----------- CREATING THE DIFFERENT CONTENT-BASED FILTERING MODELS ----------- #
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error


# Assigning the different machine learning algorithms to be implemented in the models (incl. hyperparameters) to a dictionary
ml_algorithms = {"Linear regression": LinearRegression(), "Lasso": Lasso(alpha=1.0, max_iter=10000), 
                 "KNN_7": KNeighborsRegressor(n_neighbors=7),
                 "RFR": RandomForestRegressor(n_estimators=1000, n_jobs=3, max_features='sqrt', random_state=0),
                 "SVR": SVR(C=1.0)}

# Saving lists that I later use to construct a dataframe containing the performances of the models
CBF_models_listed = []
RMSE_CBF_listed = []

# For every machine learning algorithm in the dictionary:
for name, ml_alg in ml_algorithms.items():
    
    # 모델 학습
    model = ml_alg
    model.fit(X_train, y_train)

    
    # 테스트 데이터셋에 대한 예측 수행
    y_pred = model.predict(X_test)

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"{ml_alg} RMSE: {rmse}")
model=RandomForestRegressor(n_estimators=1000, n_jobs=3, max_features='sqrt', random_state=0)